In [ ]:
!pip install mat73
!pip install entropy

In [54]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import mat73
from scipy.stats import skew, kurtosis
from scipy.signal import find_peaks
from scipy.fft import fft, fftshift
from scipy.signal import welch, lombscargle
from sklearn.metrics import mutual_info_score
from scipy.stats import skew, kurtosis, pearsonr, entropy
from scipy.signal import welch
from collections import OrderedDict
from scipy import signal
from numpy.linalg import inv, svd
from scipy.linalg import solve_toeplitz
from numpy import dot
from scipy.io import savemat

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/MyDrive/EEG_Project/Data_main/


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[Errno 2] No such file or directory: 'drive/MyDrive/EEG_Project/Data_main/'
/content/drive/MyDrive/EEG_Project/Data_main


###**Peak first PD and CLT from dataset**

In [ ]:
# Define functions to compute various features

def var_feature(data):
    return np.var(data, axis=1)  # Compute variance along the time axis

def skewness_feature(data):
    return skew(data, axis=1)  # Compute skewness along the time axis

def corr_feature(data):
    n_channels = data.shape[0]
    corr = np.zeros((n_channels, n_channels))
    for i in range(n_channels):
        for j in range(i, n_channels):
            corr_ij, _ = pearsonr(data[i,:,:].flatten(), data[j,:,:].flatten())
            corr[i,j] = corr_ij
            corr[j,i] = corr_ij
    return corr  # Compute Pearson correlation matrix of channels across time

def kurtosis_feature(data):
    return kurtosis(data, axis=1)  # Compute kurtosis along the time axis

def histogram_feature(data):
    n_bins = 10  # Define number of histogram bins
    hist_feats = np.zeros((63, 10))
    for i in range(data.shape[0]):
        hist, _ = np.histogram(data[i,:,:].flatten(), bins=n_bins)
        hist_feats[i, :] = hist
    # return np.concatenate(hist_feats, axis=0)  # Concatenate histogram features for each channel
    return hist_feats

def ar_coeff_feature(data):
    # Compute autoregressive (AR) model coefficients using Burg method
    n_coeffs = 4  # Define number of AR model coefficients to compute
    ar_coeffs = np.zeros((data.shape[0], n_coeffs))
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            r = np.correlate(data[i,j,:], data[i,j,:], mode='full')
            r = r[data[i,j,:].size - 1:]
            b = r[1:n_coeffs+1]
            r = r[:n_coeffs]
            ar_coeffs[i,:] = solve_toeplitz((r, b), r)
        ar_coeffs[i,:] /= data.shape[1]
    return ar_coeffs


def entropy_feature(data):
    return entropy(data, axis=1)  # Compute Shannon entropy along the time axis

def freq_ratio_feature(data):
    # Compute power spectral ratio of each frequency band to whole frequency band (0.5 31 Hz)
    freqs, pxx = welch(data, fs=500, nperseg=256, noverlap=128, nfft=2048, axis=1, detrend=False)
    delta_band = np.where((freqs >= 0.5) & (freqs <= 4))[0]
    theta_band = np.where((freqs >= 4) & (freqs <= 8))[0]
    alpha_band = np.where((freqs >= 8) & (freqs <= 11))[0]
    beta_band = np.where((freqs >= 11) & (freqs <= 31))[0]
    whole_band = np.where((freqs >= 0.5) & (freqs <= 31))[0]
    power_delta = np.sum(pxx[:,:,delta_band], axis=2)
    power_theta = np.sum(pxx[:,:,theta_band], axis=2)
    power_alpha = np.sum(pxx[:,:,alpha_band], axis=2)
    power_beta = np.sum(pxx[:,:,beta_band], axis=2)
    power_whole = np.sum(pxx[:,:,whole_band], axis=2)
    power_band = np.concatenate(np.concatenate(np.concatenate(power_delta, power_theta),power_alpha),power_beta)

    return power_band / power_whole

In [ ]:
Features_train = OrderedDict()
Features_train['var'] = var_feature(train_data)
Features_train['skewness'] = skewness_feature(train_data)
Features_train['corr'] = corr_feature(train_data)
Features_train['kurtosis'] = kurtosis_feature(train_data)
Features_train['hist'] = histogram_feature(train_data)
Features_train['ar_coeff'] = ar_coeff_feature(train_data)
Features_train['entropy'] = entropy_feature(train_data)
Features_train['freq_ratio'] = freq_ratio_feature(train_data)

TypeError: ignored

In [ ]:
var_feature(train_data).shape

(63, 245)

In [ ]:
skewness_feature(train_data).shape

(63, 245)

In [ ]:
corr_feature(train_data).shape

(63, 63)

In [ ]:
kurtosis_feature(train_data).shape

(63, 245)

In [ ]:
histogram_feature(train_data).shape

(63, 10)

In [ ]:
ar_coeff_feature(train_data).shape

(63, 4)

In [ ]:
entropy_feature(train_data).shape

(63, 245)

In [ ]:
freq_ratio_feature(train_data).shape

(4, 63, 1025)

In [ ]:
Feature_matrix_train = np.hstack(list(Features_train.values()))
print("Shape of feature matrix:", Feature_matrix_train.shape)

ValueError: ignored